<a href="https://colab.research.google.com/github/ricoen/google-ee-data-extraction/blob/main/ee_data_extraction_poi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine Data Extract

Author [@ricoen](https://github.com/ricoen)<br>
Please cite me if you use this code :)

In [ ]:
import ee

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
dataset = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')

In [ ]:
start_date = "2022-07-01"
stop_date = "2022-07-31"

dataset = dataset.select("Tair_f_inst").filterDate(start_date, stop_date)

In [ ]:
lon = 112.611523
lat = -7.951988

poi = ee.Geometry.Point(lon, lat)

In [ ]:
res = 27830

t_air = dataset.mean().sample(poi, res).first().get('Tair_f_inst').getInfo()
print('Average daytime air termperature at urban point:', round(t_air -273.15, 2), '°C')

In [ ]:
t_air_poi = dataset.getRegion(poi, res).getInfo()

In [ ]:
t_air_poi[:3]

In [ ]:
import pandas as pd
from datetime import datetime as dt

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    time = dataset.aggregate_array("system:time_start").getInfo()
    date = [x // 1000 for x in time]
    captured = [dt.utcfromtimestamp(i).strftime("%c GMT") for i in date]
    captured = pd.Series(captured)
    df["datetime"] = pd.to_datetime(captured).dt.tz_convert("Asia/Jakarta")

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [ ]:
t_air_df = ee_array_to_df(t_air_poi, ['Tair_f_inst'])

In [ ]:
t_air_df.head()

In [ ]:
def t_air_to_celsius(t_air):
    """Converts MODIS LST units to degrees Celsius."""
    t_celsius =  t_air - 273.15
    
    return round(t_celsius)

t_air_df['Tair_f_inst'] = t_air_df['Tair_f_inst'].apply(t_air_to_celsius)

In [ ]:
t_air_df.head()

In [ ]:
t_air_df

In [ ]:
#Visualize the data with Plotly
import plotly.express as px

#Define x and y values
fig = px.line(t_air_df, x="datetime", y="Tair_f_inst", 
              title="Air Temperature on Faculty of Mathematics and Natural Sciences, Universitas Brawijaya - GLDAS-2.1 (NASA)")
#Show the figure
fig.show()